In [7]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

student_files = [doc for doc in os.listdir() if doc.endswith('.txt')]
student_notes = [open(_file, encoding='utf-8').read() for _file in student_files]

def vectorize(Text): return TfidfVectorizer().fit_transform(Text).toarray()
def similarity(doc1, doc2): return cosine_similarity([doc1, doc2])

vectors = vectorize(student_notes)
s_vectors = list(zip(student_files, vectors))
plagiarism_results = set()

def check_plagiarism():
    global s_vectors
    for student_a, text_vector_a in s_vectors:
        new_vectors = s_vectors.copy()
        current_index = new_vectors.index((student_a, text_vector_a))
        del new_vectors[current_index]
        for student_b, text_vector_b in new_vectors:
            sim_score = similarity(text_vector_a, text_vector_b)[0][1]
            student_pair = sorted((student_a, student_b))
            score = (student_pair[0], student_pair[1], sim_score)
            plagiarism_results.add(score)
    return plagiarism_results

for data in check_plagiarism():
    print(data)


('doan_10.txt', 'doan_6.txt', 0.13146919378291513)
('doan_6.txt', 'doan_7.txt', 0.5387950480045754)
('doan_10.txt', 'doan_5.txt', 0.10399423210730158)
('doan_5.txt', 'doan_8.txt', 0.0)
('doan_2.txt', 'doan_3.txt', 0.3696460889854305)
('doan_3.txt', 'doan_4.txt', 0.5773247799261676)
('doan_1.txt', 'doan_8.txt', 0.0)
('doan_3.txt', 'doan_5.txt', 0.006275269898820604)
('doan_3.txt', 'doan_6.txt', 0.0)
('doan_4.txt', 'doan_7.txt', 0.0041216544735306745)
('doan_1.txt', 'doan_7.txt', 0.13235428146127068)
('doan_2.txt', 'doan_5.txt', 0.0)
('doan_10.txt', 'doan_4.txt', 0.0)
('doan_7.txt', 'doan_8.txt', 0.07614751065033604)
('doan_5.txt', 'doan_7.txt', 0.22342183689008713)
('doan_10.txt', 'doan_2.txt', 0.0)
('doan_10.txt', 'doan_7.txt', 0.1468259304865595)
('doan_1.txt', 'doan_10.txt', 0.837949330735135)
('doan_10.txt', 'doan_3.txt', 0.0)
('doan_5.txt', 'doan_6.txt', 0.21443240321227774)
('doan_1.txt', 'doan_6.txt', 0.10484779315683229)
('doan_4.txt', 'doan_6.txt', 0.0)
('doan_1.txt', 'doan_5.t

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from unidecode import unidecode

def vectorize(Text): 
    return TfidfVectorizer().fit_transform(Text).toarray()

def similarity(doc1, doc2): 
    return cosine_similarity([doc1, doc2])

# Đọc nội dung từ hai tệp tin văn bản
document1_text = open("doan_1.txt", encoding='utf-8').read()
# document1_text = re.sub(r'[^\w\s]', '', document1_text)
# document1_text = re.sub(r'[^\w\s]', '', unidecode(document1_text))
document2_text = open("doan_10.txt", encoding='utf-8').read()

# Chuyển đổi đoạn văn thành vector
vectors = vectorize([document1_text, document2_text])

# So sánh tương đồng giữa hai đoạn văn
similarity_score = similarity(vectors[0], vectors[1])[0][1]

print("Tương đồng:", similarity_score)


Tương đồng: 0.8585968293130968


# Test với url

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, pipeline

def vectorize(Text): 
    return TfidfVectorizer().fit_transform(Text).toarray()

def similarity(doc1, doc2): 
    return cosine_similarity([doc1, doc2])

def calculate_similarity_score_bert(text1, text2):
    # Sử dụng DistilBERT để tính toán độ tương đồng giữa hai đoạn văn
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-multilingual-cased")
    model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=1)
    similarity_analyzer = pipeline('text-classification', model=model, tokenizer=tokenizer)

    # Tính toán độ tương đồng
    similarity_score = similarity_analyzer(text1, text2)[0]['score']

    return similarity_score

def check_plagiarism_with_tfidf_bert(text1, text2, threshold_tfidf=0.8, threshold_bert=0.8):
    # Vector hóa dữ liệu bằng TF-IDF
    vectors_tfidf = vectorize([text1, text2])

    # Tính toán độ tương đồng với TF-IDF
    similarity_score_tfidf = similarity(vectors_tfidf[0], vectors_tfidf[1])[0][1]

    # Tính toán độ tương đồng với DistilBERT
    similarity_score_bert = calculate_similarity_score_bert(text1, text2)

    print("Tương đồng TF-IDF:", similarity_score_tfidf)
    # print("Tương đồng DistilBERT:", similarity_score_bert)

    # Kết hợp kết quả từ cả hai phương pháp
    combined_score = (similarity_score_tfidf + similarity_score_bert) / 2

    return combined_score >= max(threshold_tfidf, threshold_bert)

# Đọc nội dung từ hai tệp tin văn bản
document1_text = open("doan_1.txt", encoding='utf-8').read()
document2_text = open("doan_10.txt", encoding='utf-8').read()

# Kiểm tra đạo văn
is_plagiarized = check_plagiarism_with_tfidf_bert(document1_text, document2_text)
print("Is Plagiarized:", is_plagiarized)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Ignoring args : ('The development of artificial intelligence (AI) has witnessed tremendous progress in recent years. From chatbots to autonomous vehicles, AI applications have become an integral part of our daily lives. Machine learning algorithms, a subset of AI, play a crucial role in enabling computers to learn from data and make decisions without explicit programming.\n\nOne area where AI has made significant strides is natural language processing (NLP). NLP algorithms allow machines to understand, interpret, and generate human-like language. This has led to advancements in virtual assistants, language translation, and senti

RuntimeError: The size of tensor a (2629) must match the size of tensor b (512) at non-singleton dimension 1

# Test thư viện dịch của google


In [5]:
# Import thư viện deep-translator
from deep_translator import GoogleTranslator

# Định nghĩa một đoạn văn bản tiếng Việt
# Định nghĩa tên file và đường dẫn
filename = "doan_1.txt"
# filepath = "c:/users/user/documents/" + filename

# Mở file ở chế độ đọc
file = open(filename, "r")

# Đọc nội dung file và lưu vào biến text
text = file.read()

# Đóng file
file.close()

# In nội dung file
print(text)
# Dịch đoạn văn bản sang tiếng Anh bằng GoogleTranslator
translation = GoogleTranslator(source='vi', target='en').translate(text)

# In kết quả dịch
print(translation)


Hiện, việc bắn pháo hoa thực hiện theo Nghị định 362009. Vào giao thừa Tết Nguyên đán, các thành phố trực thuộc Trung ương và Thừa Thiên Huế được bắn pháo hoa nổ tầm cao và tầm thấp, thời lượng không quá 15 phút. Các tỉnh còn lại được bắn pháo hoa nổ tầm thấp không quá 15 phút.
Ngày Quốc khánh, các thành phố trực thuộc Trung ương và Thừa Thiên Huế được bắn pháo hoa nổ tầm cao và tầm thấp vào lúc 21h ngày 29 trong 15 phút.
Ngày 304, Hà Nội và TP HCM được bắn pháo hoa nổ tầm cao và tầm thấp không quá 15 phút, vào 21h.
Ngoài ra các dịp được bắn pháo hoa như ngày thành lập các tỉnh, thành phố thuộc Trung ương sự kiện văn hóa, du lịch, thể thao mang tính quốc gia, quốc tế... Những trường hợp khác, trong đó có Tết Dương lịch, do Thủ tướng quyết định.
Lê Tuyết
Currently, fireworks displays are carried out in accordance with Decree 362009. On Lunar New Year's Eve, centrally run cities and Thua Thien Hue are allowed to display high- and low-range fireworks, lasting no more than 15 minutes. The 

In [2]:
# Import thư viện deep-translator
from deep_translator import GoogleTranslator

# Nhập vào đoạn văn bản ngôn ngữ bất kì
text = input("Nhập vào đoạn văn bản ngôn ngữ bất kì: ")

# Xác định ngôn ngữ của đoạn văn bản bằng hàm detect_language
source = GoogleTranslator.detect_language(text)

# Dịch đoạn văn bản sang tiếng Anh bằng hàm translate
translation = GoogleTranslator(source=source, target='en').translate(text)

# In kết quả dịch
print("Đoạn văn bản đã được dịch sang tiếng Anh là: ")
print(translation)


# Lấy dữ liệu tưg trsng web và lưu vào tệp txt

In [1]:
import requests
from underthesea import word_tokenize
from bs4 import BeautifulSoup

# URL của trang web
url = "https://vi.wikipedia.org/wiki/T%C6%B0%E1%BB%A3ng_N%E1%BB%AF_th%E1%BA%A7n_T%E1%BB%B1_do"

# Sử dụng requests để tải nội dung trang web
response = requests.get(url)

# Kiểm tra xem việc tải nội dung thành công hay không
if response.status_code == 200:
    # Lấy nội dung HTML của trang web
    html = response.text

    # Tạo một đối tượng BeautifulSoup để phân tích nội dung HTML
    soup = BeautifulSoup(html, "html.parser")

    # Trích xuất văn bản từ trang web và loại bỏ các đoạn trống
    text = '\n'.join([line.strip() for line in soup.stripped_strings])

    # Lưu nội dung vào file
    with open("doan_10.txt", "w", encoding="utf-8") as file:
        file.write(text)

    print("Nội dung đã được lưu vào file doan_10.txt.")
else:
    print(f"Không thể tải nội dung. Mã trạng thái: {response.status_code}")


Nội dung đã được lưu vào file doan_10.txt.


# Chỉ lấy văn bản trong thẻ p. bỏ kí tự đặt biệt giữ lại câu tiếng việt và , .

In [5]:
import requests
import re
from bs4 import BeautifulSoup

# URL của trang web
url = "https://vnexpress.net/tp-hcm-xin-ban-phao-hoa-dip-tet-duong-lich-4682385.html"

# Sử dụng requests để tải nội dung trang web
response = requests.get(url)

# Kiểm tra xem việc tải nội dung thành công hay không
if response.status_code == 200:
    # Lấy nội dung HTML của trang web
    html = response.text

    # Tạo một đối tượng BeautifulSoup để phân tích nội dung HTML
    soup = BeautifulSoup(html, "html.parser")

    # Tìm tất cả các thẻ <p> và lấy nội dung từ chúng
    paragraphs = soup.find_all('p')

    # Hàm loại bỏ kí tự đặc biệt và giữ lại dấu câu
    def remove_special_characters(text):
        cleaned_text = re.sub(r'[^\w\s.,?!]', '', text)
        return cleaned_text

    # Lấy văn bản từ các thẻ <p> và loại bỏ các đoạn trống
    text = '\n'.join([remove_special_characters(paragraph.get_text()).strip() for paragraph in paragraphs])

    # Lưu nội dung vào file
    with open("doan_10.txt", "a", encoding="utf-8") as file:
        file.write(text)

    print("Nội dung đã được lưu vào file doan_10.txt.")
else:
    print(f"Không thể tải nội dung. Mã trạng thái: {response.status_code}")


Nội dung đã được lưu vào file doan_10.txt.


# lấy vưn bản thẻ p trong web rồi loại bỏ kí tự đặt biệt. Cả đấu chấm và dấu phẩy

In [3]:
import requests
import re
from bs4 import BeautifulSoup

# URL của trang web
url = "https://vi.wikipedia.org/wiki/T%C6%B0%E1%BB%A3ng_N%E1%BB%AF_th%E1%BA%A7n_T%E1%BB%B1_do"

# Sử dụng requests để tải nội dung trang web
response = requests.get(url)

# Kiểm tra xem việc tải nội dung thành công hay không
if response.status_code == 200:
    # Lấy nội dung HTML của trang web
    html = response.text

    # Tạo một đối tượng BeautifulSoup để phân tích nội dung HTML
    soup = BeautifulSoup(html, "html.parser")

    # Tìm tất cả các thẻ <p> và lấy nội dung từ chúng
    paragraphs = soup.find_all('p')

    # Hàm để loại bỏ kí tự đặc biệt, dấu câu và khoảng trắng không mong muốn
    def clean_text(text):
        # Loại bỏ kí tự đặc biệt, dấu câu và chuyển đổi thành chữ thường
        cleaned_text = re.sub(r'[^\w\s]', '', text)
        # Loại bỏ dấu câu và chuyển đổi thành chữ thường
        cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
        return cleaned_text.strip()

    # Lấy văn bản từ các thẻ <p>, loại bỏ kí tự đặc biệt, dấu câu và khoảng trắng không mong muốn
    text = '\n'.join([clean_text(paragraph.get_text()) for paragraph in paragraphs])

    # Lưu nội dung vào file
    with open("doan_10.txt", "w", encoding="utf-8") as file:
        file.write(text)

    print("Nội dung đã được lưu vào file doan_10.txt.")
else:
    print(f"Không thể tải nội dung. Mã trạng thái: {response.status_code}")


Nội dung đã được lưu vào file doan_10.txt.


In [11]:
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import re

# URL của trang web
url = "https://vi.wikipedia.org/wiki/T%C6%B0%E1%BB%A3ng_N%E1%BB%AF_th%E1%BA%A7n_T%E1%BB%B1_do"

# Sử dụng requests để tải nội dung trang web
response = requests.get(url)

# Kiểm tra xem việc tải nội dung thành công hay không
if response.status_code == 200:
    # Lấy nội dung HTML của trang web
    html = response.text

    # Tạo một đối tượng BeautifulSoup để phân tích nội dung HTML
    soup = BeautifulSoup(html, "html.parser")

    # Tìm tất cả các thẻ <p> và lấy nội dung từ chúng
    paragraphs = soup.find_all('p')

    # Hàm để loại bỏ kí tự đặc biệt, dấu câu và khoảng trắng không mong muốn, và xóa dấu tiếng Việt
    def clean_text(text):
        cleaned_text = re.sub(r'[^\w\s]', '', text)
        cleaned_text = re.sub(r'[^\w\s]', '', unidecode(cleaned_text))
        return cleaned_text.strip()

    # Lấy văn bản từ các thẻ <p>, loại bỏ kí tự đặc biệt, dấu câu, khoảng trắng không mong muốn, và xóa dấu tiếng Việt
    text = '\n'.join([clean_text(paragraph.get_text()) for paragraph in paragraphs])

    # Lưu nội dung vào file
    with open("doan_10.txt", "w", encoding="utf-8") as file:
        file.write(text)

    print("Nội dung đã được lưu vào file doan_10.txt.")
else:
    print(f"Không thể tải nội dung. Mã trạng thái: {response.status_code}")


Nội dung đã được lưu vào file doan_10.txt.


In [7]:
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import re

# URL của trang web
url = "https://vnexpress.net/"

# Sử dụng requests để tải nội dung trang web
response = requests.get(url)

# Kiểm tra xem việc tải nội dung thành công hay không
if response.status_code == 200:
    # Lấy nội dung HTML của trang web
    html = response.text

    # Tạo một đối tượng BeautifulSoup để phân tích nội dung HTML
    soup = BeautifulSoup(html, "html.parser")

    # Tìm tất cả các thẻ <p> và lấy nội dung từ chúng
    paragraphs = soup.find_all('p')

    # Hàm để chỉ loại bỏ kí tự đặc biệt và xóa dấu tiếng Việt
    def clean_text(text):
        cleaned_text = re.sub(r'[^\w\s.,?!]', '', unidecode(text))
        return cleaned_text.strip()

    # Lấy văn bản từ các thẻ <p>, loại bỏ kí tự đặc biệt, và xóa dấu tiếng Việt
    text = '\n'.join([clean_text(paragraph.get_text()) for paragraph in paragraphs])

    # Lưu nội dung vào file
    with open("doan_10.txt", "w", encoding="utf-8") as file:
        file.write(text)

    print("Nội dung đã được lưu vào file doan_10.txt.")
else:
    print(f"Không thể tải nội dung. Mã trạng thái: {response.status_code}")


Nội dung đã được lưu vào file doan_10.txt.
